In [1]:
!wget "https://www.dropbox.com/scl/fi/lqhh5awvhwd1nyqdd76vu/data.zip?rlkey=7knxpv2bs9kxxm3zmwd92vjr3&st=678q4o8s&dl=1" -O data.zip
!apt update && apt install -y unzip
!unzip data.zip -d data

--2025-06-18 13:06:57--  https://www.dropbox.com/scl/fi/lqhh5awvhwd1nyqdd76vu/data.zip?rlkey=7knxpv2bs9kxxm3zmwd92vjr3&st=678q4o8s&dl=1
Resolving www.dropbox.com (www.dropbox.com)... 162.125.64.18, 2620:100:6020:18::a27d:4012
Connecting to www.dropbox.com (www.dropbox.com)|162.125.64.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://uca4b7a6a28d914f76c91a2f774a.dl.dropboxusercontent.com/cd/0/inline/Cr3TVWbCKqrWwz7BNGFxVbv7CjwPC6le5y6duHn_BmxQKwNfMus-Pxo0zc8Mt14mHAxM7oMBUG46XgghibFSb-Ks6oZXbj1eVvzW7DGql1RXzJg2sElx8g_lI-sD6mHY7J3Pyb6xyzWGpKyWfBH7Lxt3/file?dl=1# [following]
--2025-06-18 13:06:57--  https://uca4b7a6a28d914f76c91a2f774a.dl.dropboxusercontent.com/cd/0/inline/Cr3TVWbCKqrWwz7BNGFxVbv7CjwPC6le5y6duHn_BmxQKwNfMus-Pxo0zc8Mt14mHAxM7oMBUG46XgghibFSb-Ks6oZXbj1eVvzW7DGql1RXzJg2sElx8g_lI-sD6mHY7J3Pyb6xyzWGpKyWfBH7Lxt3/file?dl=1
Resolving uca4b7a6a28d914f76c91a2f774a.dl.dropboxusercontent.com (uca4b7a6a28d914f76c91a2f774a.dl.dropboxusercontent.co

In [5]:
!pip install --upgrade transformers
!pip install wandb
!pip install -U "accelerate>=0.26.0"
!pip install torch==2.5.1 transformers==4.49.0
!pip install huggingface_hub transformers
!pip install scikit-learn

  Using cached transformers-4.52.4-py3-none-any.whl.metadata (38 kB)
Using cached transformers-4.52.4-py3-none-any.whl (10.5 MB)
  Attempting uninstall: transformers
    Found existing installation: transformers 4.49.0
    Uninstalling transformers-4.49.0:
      Successfully uninstalled transformers-4.49.0

[notice] A new release of pip is available: 23.3.1 -> 25.1.1
[notice] To update, run: python -m pip install --upgrade pip

[notice] A new release of pip is available: 23.3.1 -> 25.1.1
[notice] To update, run: python -m pip install --upgrade pip

[notice] A new release of pip is available: 23.3.1 -> 25.1.1
[notice] To update, run: python -m pip install --upgrade pip
  Using cached transformers-4.49.0-py3-none-any.whl.metadata (44 kB)
Using cached transformers-4.49.0-py3-none-any.whl (10.0 MB)
  Attempting uninstall: transformers
    Found existing installation: transformers 4.52.4
    Uninstalling transformers-4.52.4:
      Successfully uninstalled transformers-4.52.4

[notice] A new

In [ ]:
from transformers import ViTForImageClassification, ViTFeatureExtractor, TrainingArguments, Trainer
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import torch
from sklearn.metrics import accuracy_score
import wandb
import wandb

wandb.login(key=os.getenv("WANNDB_TOKEN"))

wandb.init(project="vit-accuracy", name="run-with-accuracy")

model_name = "google/vit-base-patch16-224"
feature_extractor = ViTFeatureExtractor.from_pretrained(model_name)

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=feature_extractor.image_mean, std=feature_extractor.image_std),
])

train_dataset = datasets.ImageFolder("data/train", transform=transform)
val_dataset = datasets.ImageFolder("data/val", transform=transform)

num_labels = len(train_dataset.classes)
model = ViTForImageClassification.from_pretrained(
    model_name,
    num_labels=num_labels,
    ignore_mismatched_sizes=True
)

def collate_fn(batch):
    return {
        "pixel_values": torch.stack([x[0] for x in batch]),
        "labels": torch.tensor([x[1] for x in batch])
    }

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = logits.argmax(-1)
    return {
        "accuracy": accuracy_score(labels, preds)
    }

training_args = TrainingArguments(
    output_dir="./vit_output",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    num_train_epochs=5,
    logging_dir="./logs",
    report_to="wandb",
    logging_steps=10,
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    greater_is_better=False,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    data_collator=collate_fn,
    tokenizer=feature_extractor,
    compute_metrics=compute_metrics
)

trainer.train()


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: strauchjustin30 (strauchjustin30-pagemachine) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


preprocessor_config.json:   0%|          | 0.00/160 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/models/vit/feature_extraction_vit.py:28: FutureWarning: The class ViTFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use ViTImageProcessor instead.
  warnings.warn(


config.json:   0%|          | 0.00/69.7k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/346M [00:00<?, ?B/s]

Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224 and are newly initialized because the shapes did not match:
- classifier.bias: found shape torch.Size([1000]) in the checkpoint and torch.Size([9]) in the model instantiated
- classifier.weight: found shape torch.Size([1000, 768]) in the checkpoint and torch.Size([9, 768]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1594: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/tmp/ipykernel_233/4145328738.py:60: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
wandb: WARNING The `run_name` is currently set to 

Epoch,Training Loss,Validation Loss,Accuracy
1,0.016500,0.072679,0.980210
2,0.000900,0.039496,0.990687
3,0.000400,0.044351,0.990687
4,0.000100,0.053679,0.989523
5,0.000100,0.054440,0.988359


TrainOutput(global_step=4265, training_loss=0.03686344784506103, metrics={'train_runtime': 1623.4854, 'train_samples_per_second': 20.995, 'train_steps_per_second': 2.627, 'total_flos': 2.6414801788760986e+18, 'train_loss': 0.03686344784506103, 'epoch': 5.0})

In [7]:
trainer.save_model("./vit_output/best_model")

In [9]:
import shutil
import random
from pathlib import Path
from collections import Counter
import json

from transformers import ViTForImageClassification, ViTImageProcessor
import torch
from PIL import Image
from torchvision import transforms
from tqdm import tqdm

test_dir = Path("./data") / "test"
image_extensions = {".jpg", ".jpeg", ".png", ".bmp", ".gif", ".tiff", ".webp"}
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
eq_len_override = None

model = ViTForImageClassification.from_pretrained("JustinStrauch/vit_fit_v3")
processor = ViTImageProcessor.from_pretrained("JustinStrauch/vit_fit_v3")
model.to(device)
model.eval()

try:
    with open("class_to_idx.json") as f:
        idx_to_class = {v: k for k, v in json.load(f).items()}
except FileNotFoundError:
    idx_to_class = {}

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=processor.image_mean, std=processor.image_std),
])

prediction_cache = {}

def predict(image_path: str) -> str:
    if image_path in prediction_cache:
        return prediction_cache[image_path]
    
    image = Image.open(image_path).convert("RGB")
    input_tensor = transform(image).unsqueeze(0).to(device)

    with torch.no_grad():
        outputs = model(input_tensor)
        predicted_idx = outputs.logits.argmax(dim=1).item()

    predicted_label = subfolders[int(idx_to_class.get(predicted_idx, str(predicted_idx)))]
    prediction_cache[image_path] = predicted_label
    return predicted_label

def get_prediction(path, label, test_label):
    result = [predict(str(path))]
    if label == test_label:
        if not test_label in result:
            print(f"for {path}: {label} -> {result}")
        return test_label in result
    else:
        if test_label in result:
            print(f"for {path}: {label} -> {result}")
        return not test_label in result

def get_precision(file_list, label):
    positiv = {}
    for file in tqdm(file_list, desc=f"Testing label: {label}"):
        file_label = str(file).split("/")[-2]
        positiv[file_label] = positiv.get(file_label, 0) + int(get_prediction(file, file_label, label))
    return positiv

def equalise(seq, label, eq_len):
    seq_list = {}
    for x in seq:
        x_label = str(x).split("/")[-2]
        seq_list[x_label] = seq_list.get(x_label, []) + [x]

    if eq_len_override is not None:
        eq_len = eq_len_override
    else:
        eq_len = min(len(v) for v in seq_list.values())

    result = []
    for x in seq_list:
        random.shuffle(seq_list[x])
        result += seq_list[x][:eq_len]

    return result

subfolders = sorted([str(f).split("/")[-1] for f in test_dir.iterdir() if f.is_dir()])
files = [f for f in test_dir.rglob("*") if f.is_file() and f.suffix.lower() in image_extensions]

all_labels = 0
for sub in subfolders:
    sub_dir = test_dir / sub
    files = [f for f in sub_dir.rglob("*") if f.is_file() and f.suffix.lower() in image_extensions]
    tmp = get_precision(files, sub)
    this_labels = 0
    prec = tmp[sub] / len(files)
    all_labels += prec
    print(f"Accuracy für {sub}: {prec:.3f}")

print(f"\nGesamt-Accuracy über alle Klassen: {all_labels / len(subfolders):.3f}")


config.json:   0%|          | 0.00/987 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/343M [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/351 [00:00<?, ?B/s]

Testing label: chart:  20%|██        | 30/150 [00:01<00:03, 31.16it/s]

for data/test/chart/bar_chart_9f9bc3d4-5be9-46dd-ae87-df2caf4d7a9b.jpg: chart -> ['infographic']


...

for data/test/tabel/Aspiriini_vesi_Spartan_lukio_2.png: tabel -> ['chart']


Testing label: tabel: 100%|██████████| 100/100 [00:04<00:00, 22.85it/s]


Accuracy für tabel: 0.990


Testing label: unrealistic: 100%|██████████| 100/100 [00:04<00:00, 21.40it/s]

Accuracy für unrealistic: 1.000

Gesamt-Accuracy über alle Klassen: 0.990


In [ ]:
from huggingface_hub import login
from transformers import ViTForImageClassification, ViTImageProcessor

login(token=os.getenv("HUGGING_FACE_TOKEN"))

model = ViTForImageClassification.from_pretrained("./vit_output/best_model")
processor = ViTImageProcessor.from_pretrained("./vit_output/best_model")

repo_id = "JustinStrauch/vit_fit_v3"

model.push_to_hub(repo_id)
processor.push_to_hub(repo_id)

model.safetensors:   0%|          | 0.00/343M [00:00<?, ?B/s]

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/JustinStrauch/vit_fit_v3/commit/747aeaddb85e82de2918021bc20c4408b57b3177', commit_message='Upload processor', commit_description='', oid='747aeaddb85e82de2918021bc20c4408b57b3177', pr_url=None, repo_url=RepoUrl('https://huggingface.co/JustinStrauch/vit_fit_v3', endpoint='https://huggingface.co', repo_type='model', repo_id='JustinStrauch/vit_fit_v3'), pr_revision=None, pr_num=None)